In [1]:
from pyspark import SparkContext, SparkConf
from pyspark.sql.types import *

conf = SparkConf().setAppName("preprocess").setMaster("local")
sc = SparkContext.getOrCreate(conf)
spark = SparkSession.builder.master("local").appName("preprocess").getOrCreate()

In [2]:
from pyspark.mllib.util import Vectors
from pyspark.mllib.linalg import VectorUDT
from pyspark.sql.functions import UserDefinedFunction
from pyspark.sql.types import DataType, StringType

def output_csv(df, path):
    udf = UserDefinedFunction(lambda x: Vectors.stringify(x), StringType())
    new_df = df.withColumn('features', udf(df.features))
    
    new_df.write.csv(path, header=True)
    
def read_csv(path):
    df = spark.read.csv(path, header=True, inferSchema=True)
    
    udf = UserDefinedFunction(lambda x: Vectors.parse(x), VectorUDT())
    new_df = df.withColumn('features', udf(df.features))
    
    return new_df

In [4]:
testdf = read_csv("./data/DATA_TFIDF_HADM_TOP10.csv")
print testdf.count()
testdf.show()

40562
+------+----+----+-----+----+-----+-----+-----+-----+----+----+--------------------+
|    id|4019|2724|25000|4280|41401|53081|51881|42731|5849|5990|            features|
+------+----+----+-----+----+-----+-----+-----+-----+----+----+--------------------+
|117760|   0|   0|    0|   0|    0|    1|    1|    0|   0|   0|(40000,[69,372,69...|
|129030|   1|   1|    0|   0|    0|    1|    0|    0|   0|   0|(40000,[13,32,83,...|
|172040|   0|   0|    0|   0|    1|    0|    0|    0|   1|   0|(40000,[10,69,152...|
|156170|   0|   0|    1|   1|    0|    0|    0|    1|   1|   0|(40000,[3,78,130,...|
|199180|   0|   0|    1|   1|    1|    0|    0|    0|   0|   0|(40000,[48,62,80,...|
|167440|   0|   0|    1|   0|    1|    0|    1|    0|   0|   0|(40000,[207,264,2...|
|178710|   0|   0|    0|   0|    0|    0|    0|    0|   1|   0|(40000,[574,794,1...|
|162840|   0|   1|    0|   1|    0|    0|    0|    1|   0|   0|(40000,[32,130,15...|
|189980|   1|   0|    1|   1|    0|    0|    0|    0|   0| 